In [1]:
from scipy.io import loadmat
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import tkinter 
from sklearn.metrics import accuracy_score

from userDefFunc import my_function
from userDefFunc import f_logVar
import pickle

from sklearn import svm
from sklearn.model_selection import cross_val_score

subList = ["P01", "P02", "P03", "P04","P05", "P06", "P07","P08", "P09", "P10"]

info = np.empty(10)



In [2]:
dataSRC = "/media/andreas/Andreas/Dissertation/EEG data/"
    
subID = subList[0]
    
fnameWithPath=dataSRC+"parsed_"+subID+"E.mat"

print(fnameWithPath)

subDataFile = loadmat(fnameWithPath)
print (subDataFile)

arraysize = len(subDataFile)
print (arraysize)





/media/andreas/Andreas/Dissertation/EEG data/parsed_P01E.mat
{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Jan  2 14:52:57 2020', '__version__': '1.0', '__globals__': [], 'RawEEGData': array([[[ -57.08504105,  -39.52176666,   47.09091187, ...,
          -31.89426041,   71.15451813,   94.39797211],
        [ -67.40719604,  -57.97138977,   32.7985611 , ...,
          -35.82240677,   64.16590118,   84.41113281],
        [ -67.45669556,  -53.24325562,   33.25360107, ...,
          -31.47397995,   64.22527313,   77.26131439],
        ...,
        [ -93.96321869,  -72.20977783,   46.39987564, ...,
          -41.10143661,   75.86680603,  111.64434052],
        [ -86.61682892,  -57.23156738,   54.17978668, ...,
          -37.54723358,   59.26449585,   81.89395905],
        [ -82.2109375 ,  -46.25338364,   84.26654816, ...,
          -37.24626923,   81.06242371,  116.22688293]],

       [[  63.11524582,  -15.08893394,  -79.12770844, ...,
          -90.51954651,  -22.6

In [3]:


for sb in range(10):
    #   for subID in subList:
    subID=subList[sb]

    dataSRC = "/media/andreas/Andreas/Dissertation/EEG data/"
    dataSRC1="E:\\Learning_python_tkinter\\"
    fnameWithPath=dataSRC+"parsed_"+subID+"E.mat"
    fnameWithPath1=dataSRC1+"trainedModel"+subID

    print(fnameWithPath)
    print(fnameWithPath1)

    subDataFile = loadmat(fnameWithPath)
    RawEEGData = subDataFile["RawEEGData"]
    Labels = subDataFile["cueAt"]
    sampRate = subDataFile["sampRate"]

    dim=RawEEGData.shape

    nOfTrials=dim[0] #getting the number of trials
    #print(nOfTrials)
    nOfChann=dim[1]  #getting the number of channels
    #print(nOfChann)
    nOfSampsInTrial=dim[2] #getting the number of samples per trial
    #print(nOfSampsInTrial)

    with open(fnameWithPath1, 'rb') as f1:
       rd = pickle.load(f1)
    #  print(rd[4])

    svmMdl = rd[0]
    wCSP_mu = rd[1]
    wCSP_beta = rd[2]
    muBand = rd[3]
    betaBand = rd[4]

    #print(svmMdl)
    #print(wCSP_mu)
    #print(wCSP_beta)
    #print(muBand)
    #print(betaBand)

    order = 4

    mu_B, mu_A = signal.butter(order, muBand, 'bandpass', analog=False)
    beta_B, beta_A = signal.butter(order, betaBand, 'bandpass', analog=False)

    muRawEEGData=np.empty((nOfTrials, nOfChann, nOfSampsInTrial))
    betaRawEEGData=np.empty((nOfTrials, nOfChann, nOfSampsInTrial))

    for trlIndex in range(nOfTrials):

        sig=RawEEGData[trlIndex,:,:]
    #print("Properties:")
    #print(type(sig))
    #print(sig.shape)
        mu_temp=signal.lfilter(mu_B, mu_A, sig,1)
        beta_temp=signal.lfilter(beta_B, beta_A, sig,1)

        muRawEEGData[trlIndex,:,:]= mu_temp #altering the dimensions
        betaRawEEGData[trlIndex,:,:]= beta_temp #altering the dimensions
        #print(trlIndex)
        #print(muRawEEGData.shape)
        #print(betaRawEEGData.shape)

    labelsCls1 = np.where(Labels == 1)
    labelsCls1=labelsCls1[0]
    labelsCls2 = np.where(Labels == 2)
    labelsCls2=labelsCls2[0]

    feat = np.empty((nOfTrials,4))

    for trlIndex in range(nOfTrials):

        muTemp=muRawEEGData[trlIndex,:,:]
        betaTemp=betaRawEEGData[trlIndex,:,:]

        mu_temp = np.matmul(wCSP_mu,muTemp);  #calculating the Z matrix for mu band
        beta_temp = np.matmul(wCSP_beta,betaTemp);   #calculating the Z matrix for beta band

        logVarMu=f_logVar(mu_temp);      #calculating the logvariance for Mu
        logVarBeta=f_logVar(beta_temp);  #calculating the logvariance for Beta

        feat[trlIndex, :]=[logVarMu[0],  logVarMu[len(logVarMu)-1],  logVarBeta[0],  logVarBeta[len(logVarBeta)-1]]

    Test_X = feat
    Test_Y = Labels[:,0]

    Train_X = Test_X
    labelsCls1 = np.where(Labels == 1)
    labelsCls1=labelsCls1[0]
    labelsCls2 = np.where(Labels == 2)
    labelsCls2=labelsCls2[0]


    Test_Y_predicted = svmMdl.predict(Test_X)

    testAcc = accuracy_score(Test_Y, Test_Y_predicted)
    print(testAcc)
    info[sb]=testAcc


    #fbf=0
    #sbf=1
    #x1 = Train_X[labelsCls1,fbf]
    #y1 = Train_X[labelsCls1,sbf]

    #x2 = Train_X[labelsCls2,fbf]
    #y2 = Train_X[labelsCls2,sbf]

    #fig=plt.figure()
    #ax=fig.add_axes([0,0,1,1])
    #ax.scatter(x1, y1, color='b')
    #ax.scatter(x2, y2, color='r')
    #ax.set_xlabel('Grades Range')
    #ax.set_ylabel('Grades Scored')
    #ax.set_title('scatter plot')
    #plt.show()

print("meanCVaccAllSub", np.mean(info))








/media/andreas/Andreas/Dissertation/EEG data/parsed_P01E.mat
E:\Learning_python_tkinter\trainedModelP01


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Learning_python_tkinter\\trainedModelP01'